In [1]:
# review -> 이모티콘/자음 삭제 -> (띄워쓰기 + 맞춤법) -> 문장 분리
# - 문장 단위: 문장 부호, 종결어미, 연결조사로 사전 만들기

In [59]:
import pandas as pd
from kiwipiepy import Kiwi
from emoji import core
import re
import itertools

In [3]:
data = pd.read_csv('./data/train_data_1.csv', encoding='utf-8-sig')
data.head()

,review
0,맛있어요~
1,만족도 최상의 커피맛집
2,찐 커피맛집ㅎㅎ
3,로마노 진짜 달달하고 맛있고 아이스크림라떼도 꼬소하고 달달하고 진짜 맛있어요!! 마...
4,커조남 커피는 힐링입니다 소금빵도 맛있어요


In [4]:
kiwi = Kiwi()

In [10]:
# 사용자 사전 추가
kiwi.add_user_word('맛집', 'NNG', 0)
kiwi.add_user_word('라떼', 'NNG', 0)
kiwi.add_user_word('디카페인', 'NNG', 0)
# kiwi.add_user_word('카페명', 'NNP', 0) # 카페 이름 리스트가 있으면 추가

False

In [34]:
# 정규표현식 컴파일 
noun = re.compile(r'^N') # 명사형 태그
sp_char = re.compile(r'^(SS)|SW|SP|SO') # 특수문자 태그

In [57]:
# 리뷰 문장 단위로 분리
def revToSent(review):
    tmp = []
    sentences = []
    preTag = ''
    preForm = ''
    sents = kiwi.split_into_sents(review, return_tokens=True, normalize_coda=True)
    for sent in sents:
        toks = sent.tokens
        for tok in toks:
            if bool(sp_char.match(preTag)) and bool(sp_char.match(tok.tag)):
                preTag = tok.tag
                preForm = tok.form
                continue
            
            elif bool(sp_char.match(tok.tag)):
                sentences.append(kiwi.join(tmp))
                tmp.clear()
                
                preTag = tok.tag
                preForm = tok.form
                continue
            
            condition = (preTag == 'EC') and (preForm in ['고', '지만', '은데'])
            if condition and bool(noun.match(tok.tag)):
                sentences.append(kiwi.join(tmp))
                tmp.clear()

            tmp.append(tok)
            preTag = tok.tag
            preForm = tok.form            
            
        connective = (preTag == 'EF') and (preForm in ['던데', '는데', '은데'])
        if not connective and len(tmp) > 0:
            sentences.append(kiwi.join(tmp))
            tmp.clear()
    
    return sentences

In [ ]:
preprocess = []

for review in data['review']:
    # print('원본 문장: ', review)
    review = core.replace_emoji(review, replace="")
    # print('특수 문자 제거: ', review)
    sents = revToSent(review)
    preprocess.append(sents)
    # print('문장 분리: ', sents)
    # print()    

In [ ]:
preprocess = list(itertools.chain(*preprocess))

In [ ]:
df = pd.DataFrame('reviews': preprocess)
df.to_csv('./data/train_x.csv', index=False)